<a href="https://colab.research.google.com/github/ktichola/Image-matching/blob/main/%F0%9F%93%B8_IMC_with_rotation_on_match_pair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-matching-challenge-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71885%2F8143495%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240515%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240515T085953Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1eeb2152d1864b22c560089328e39b04229145160c8a79967bb96266396882810e9532d6d8b78404a09197a6fec7313d794bf594a18dd61fe8219cc4f06cd54dff5b40683c098738b81aeb75af7459fe37966dcb55a87199c13e3f3823df663338a430c4c72cb265b852f56486a405243b8865ccbaf47d70de688eaf06e512e6d688b4808a4c99e7190298723dd9f6e3e0a94989ca6285931f0360391291cd1d5c81c75ef1b5bee3fd1f63babb1e6e1e6a7c5b84028133040207e06b32c5e3841d238d28b3d0330b036aaf27770ac7512119f70464dd46c1b83012787999c3dc792c50eb37c356cb7e090ca9c5a9701b85ee8b1b0c99272d5c8e402bd5ad0fa9,imc2024-packages-lightglue-rerun-kornia:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628051%2F7884485%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240515%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240515T085953Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1147bd3b455764b305c9c9a88ff5106210f623a632a9cb480a920f67fa2574038f5063370ea4382d55c4ee92580f0baeefa6c94e69db53efafa42fb275bd1aae2d9642c021436cf26033c8f97a61b3aa979a007c08f765206c3cded532293a29964e9d4dd11a9df7f47e77d9d72c78a8ea644fb53b438991bb13b65cd23b92710c502ddc9515a669995453157a6f53dfc5ef13af10412a57e0628675dec144009ce9747ca8e18e06d33b0241c6e384a09ed16ab0986fe37a17c93d781a907f897c4b9db3d693e9bd7daeb0baafa06b624cabff9ec4b980c420f87509700deb08a17c44f443f887604b8775b2da87c5555eb263a0e6c498418fe2ca5db1f38b88,colmap-db-import:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628331%2F7884725%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240515%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240515T085954Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2637b550f9e3d0df2f4a953a16e04fa7a5ca1ac280e00bf0fc6c07953193ab9a4ba3dd9ad70eac0901208c8d20f03dd9bae42bac219b358506daec6c5ce6e20d002c1ea957382ccb54ec5a0b0d154f7d10ba01010efc83cb93f7ed05f115b7572770173ce31cfa84c4a7a38b0448c2ba287e275d98f2416a82e64056991f1eb7ba2abed49d82fb0003bcc864cb611fb02b81bbd848215bd9c1908b6fd272672fb9d5b284adc9caf6ef4a1b13b1439887c2ca2c4cbc40d4bab8a27d66efa7c260d406550880e9c5869843684277b7f7c421adff2cfd82556306527a78de85ea445e1063c93ff77c80118bc3332fa5a946f41b971aef2af8f816d4ee4979cc2a0c,mediapy:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4973903%2F8367334%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240515%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240515T085954Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7e98eaeb5595a02881b4ee24d7ca023b7d29623e5fc400c92afec0922478b5a96c843a4e4712683bcfce8030656f7d0e051558c792641782ee5dc29262dfce121b40c9194ae5d3cd54d691d7ba6d5a77362a86a5260773c95f29f514bcb86f6214b4fadd78f8202b8b5377cbb3b81f9e002f51dc25ccfce25cbcc4776742603add8c2f310cc9a256a20cbfb589b948aebb2f6ec7e0ea73c6669b4c44c3cfc9d6f00e87f9c999921fcc9129ca90e8b1f99ef5acfb4e4a9a875fb711b2551c48c68e2e3ba94da94b4eaf1d5389223eadff4057ed99836d549c3c09849cf31c934a4acfc0e87c2677714d16db9421227164a07609d2bfcf4798f8ffd0fb168d6ae6,check-orientation:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4974117%2F8367624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240515%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240515T085954Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da4a89e2f5e46e3e8a85b663489d0c441683a86c5a8cebc0fc5d27456de92b055bc831382422fcae5805c33c32d880388af811333f8ef0db26b604ea51fea43831d8f40155f2abee81672f594e3e085320ecf1c9d1f4795bccbf37178fe93091fb157e48f3150578bba4c50c80e1e6cd11d50acb2099fe357f0711cf1fc83cd5be07f81e0712e1f7151978c761f82ab3e4fbfe20eae60e92c0e7d18b19a950bc44978e0039847346b3d96dd0e31a94f1d2ca6195371e6e654467312345bc6848bdf10d580b17101c2c5628c62ac950ee98cba318f959243adc0fcec6a0b997b2411e8c4f4d8ef2588e1fff2c0b5b43ba60e94861a523ca2da475c4040bc808dad,dinov2/pytorch/base/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3326%2F4534%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240515%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240515T085954Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9a05c6363a548072010f0c6f327b777339084c17d76e5ca3e4432522e83888ec23f9adc8449212d2dc4014dacf0aadf9bbe047d16d2c334a348ffa7ee5e078a7a6b38d7718b06b2f82d506381cc7085a9efafa8d1275696767bad16a7c8c594952f7983ca7329537b94cac6c51059c31cc0330e306d22c57163f331574118b9ef41878e8e41695e6718904a2417f3f6ca35c56247c0b881995755e081abb744e7cd87487bc54ae0489017085a334c42bdb48c9bd0c916610fb574f2d650ced93fcd041b5a71eb6502eb7356581bb7491a67e1b72459557d50221d7f19fd3daf0e6a3b651d6847bbf121aefdc23a1a4aab6e5ceccdf87016e8d2248b9a4b2a9d2,lightglue/pytorch/aliked/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14317%2F17191%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240515%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240515T085954Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9eaedec1dcc70334cb71c46049d9888f7e5c9a19697c48e2f04245b387ca9caba90e0dd5e12741cf83dd0b6dabac7b18d05be6252a4fc33adbcdee08d11eb7cd7304996b2b193a3d1e69c34a91344903fcf3853f73faed2c6acbc639c6faf42e7232d2f6514bc3eeee775dca627128e3ef3cecc2658f3ea4440cc1936bba21642c84c2cc3ee222804b40b5160cd950ee7012f181558feb57669d13fc75ba50e8aeadbafa50d2f5401d58c9c5aa7ccbd8f5d45ef64e04f2608813aeb7f37a2a392984ba4791d75434bee3f8e4b4e761a3e17336c35aed929486e88052edf62cb92956e71c304c20b8a04f6bcc7d5302048706dfd6cf0c7e29c665289fce02bb13,aliked/pytorch/aliked-n16/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14611%2F17555%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240515%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240515T085954Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4e6964051ba2706cc5b8f9f0d9efa90f2d7783a7b31b6a894de4c69bee69fbf3b6b38daf77c6f4eeff48f3f65d53336cae3de61e52156a1642b8c7f8ab0e948a5399b93f21eeb19e75a60c9dacd6744ef2ef695b813992ababb97b67957d0b9839389523b32838d5c7b2b966a2e0069341811f8dcf30043004477b3bc191d418c96887ff5e20de0a583966a98513316f4562a9cbc5c8c9ca2a6db14747bd9c89361bb1f76efb6ab25efa43223e931d26da258b276f3105e5d907c0899266a8b6a742092e4d5667e4765f870c0e056db6b0f9ae821dbb866a632f02874c7fe1e4bc3489752f2c73a390fcf91c6fe75fa7f326cec42c81e842e9db44e00955ab55,swsl_resnext50_32x4d/pytorch/v1/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F38394%2F45794%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240515%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240515T085954Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9d959d27493b3b9d0e061f1381a501a7a66c195ce37fd432ea3aa4a6c0825a1439bea262fa75ac33b612831507aabbfc5e828f5dd59c8cacad3d819afe29d9757f9fbcc23852e0a9c64ff8a42303213a6f119af124dbd94cf8789a6522715fea5d09fbbe8bf74777a89ab2478e4e62d3ec655954446ad9594ea51845181f0633e8e5e50d1e92785a48e45acc05488d9d6e184c29f225ee113009fea2ca18e47da278aa1a8600f249cc900be8d9f235603d569c6a4328e104cc97030b83460db4e2216f44a0fb6317477d7095c6f71dd09126d5c15bbb83cfd2ad7fd2d102b3f383a6341a133ac36e2552ea6ff805c490f991b12b4d6f2525a745d20233e88262'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# <center style="font-family: consolas; font-size: 32px; font-weight: bold;"> 📸 Image Matching Challenge - 📊 Understanding the baseline</center>
<p><center style="color:#949494; font-family: consolas; font-size: 20px;">Reconstruct 3D scenes from 2D images over six different domains</center></p>

***

In this notebook I explain the baseline solution provided by the organizers in [this notebook](https://www.kaggle.com/code/oldufo/imc-2024-submission-example).

I have made the code a bit easier to read, adding comments and type annotations to make it easier for you to understand what is going on.

Hope you enjoy ❤️

# Structure from Motion

Structure from Motion (SfM) is the name given to the procedure of **reconstructing a 3D scene and simultaneously obtaining the camera poses of a camera w.r.t. the given scene**. This means that, as the name suggests, we are creating the entire rigid structure from a set of images with different view points (or equivalently a camera in motion).

In this competition, the important aspect of SfM we are interested in is *obtaining the camera poses* of where each image was taken, described by a rotation matrix and translation vector from the origin. These are the objects that will be scored in our submission!

<center><img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.ENP48SmZHwG3r3O0lUVcWAHaFf%26pid%3DApi&f=1&ipt=550bf79efa85e7af870dd2d0a16793af7f3f83a36c14a1f4a648659a384b5a98&ipo=images" alt="Structure from motion structure: multiple cameras pointing toward an object in different positions and rotations that we need to find."></center>

# Baseline solution steps
In order to be able to estimate the camera poses, the solution provided by the organizers consists in the following steps:

* [1. Find pairs of images that are similar](#1)
* [2. Compute image keypoints](#2)
* [3. Match keypoints between images](#3)
* [4. Outlier detection with RANSAC](#4)
* [5. Sparse reconstruction](#5)

Let's understand how these steps are carried out

# Installing & importing relevant packages and models

In [ ]:
!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

##############ADDED######################
!pip install --no-index --find-links=/kaggle/input/mediapy/mediapy mediapy
!pip install --no-index /kaggle/input/check-orientation/check-orientation/iglovikov_helper_functions-0.0.53-py2.py3-none-any.whl --no-deps
#######################################

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
  Attempting uninstall: kornia
    Found existing installation: kornia 0.7.1
    Uninstalling kornia-0.7.1:
      Successfully uninstalled kornia-0.7.1
Looking in links: /kaggle/input/mediapy/mediapy
Processing /kaggle/input/mediapy/mediapy/mediapy-1.2.0-py3-none-any.whl
Processing /kaggle/input/check-or

In [ ]:
# General utilities
import matplotlib.pyplot as plt

import os
from tqdm import tqdm
from pathlib import Path
from time import time, sleep
from fastprogress import progress_bar
import gc
import numpy as np
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from typing import Any
import itertools
import pandas as pd

#########ADDED###########
import mediapy as media
import cv2
from glob import glob
from pprint import pprint
import shutil

from collections import namedtuple
from typing import Optional
from iglovikov_helper_functions.dl.pytorch.utils import rename_layers
from timm import create_model as timm_create_model
from torch import nn
###################

# CV/MLe
import cv2
import torch
from torch import Tensor as T
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image
from transformers import AutoImageProcessor, AutoModel

import torch
from lightglue import match_pair
from lightglue import LightGlue, ALIKED
from lightglue.utils import load_image, rbd

# 3D reconstruction
import pycolmap

# Data importing into colmap
import sys
sys.path.append("/kaggle/input/colmap-db-import")

# Provided by organizers
from database import *
from h5_to_db import *

def arr_to_str(a):
    """Returns ;-separated string representing the input"""
    return ";".join([str(x) for x in a.reshape(-1)])

def load_torch_image(file_name: Path | str, device=torch.device("cpu")):
    """Loads an image and adds batch dimension"""
    img = K.io.load_image(file_name, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

device = K.utils.get_cuda_device_if_available(0)
print(device)

DEBUG = len([p for p in Path("/kaggle/input/image-matching-challenge-2024/test/").iterdir() if p.is_dir()]) == 2
DEBUG = False
print("DEBUG:", DEBUG)

2024-05-15 08:53:08.477014: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 08:53:08.477113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 08:53:08.607147: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


cuda:0
DEBUG: False


In [ ]:
JEANS = True

# Import Check Orientation model
Input an image

Output a prediction in 4 rotation class [0deg 90deg 180deg 270deg]

using the MODELS at /kaggle/input/swsl_resnext50_32x4d/pytorch/v1/1

In [ ]:
def create_model(model_name: str, activation: Optional[str] = "softmax") -> nn.Module:
    # Define the 'model' namedtuple
    model = namedtuple("model", ["url", "model"])

    # Define the models dictionary
    models = {
        "swsl_resnext50_32x4d": model(
            model=timm_create_model("swsl_resnext50_32x4d", pretrained=False, num_classes=4),
            url="/kaggle/input/swsl_resnext50_32x4d/pytorch/v1/1/2020-11-16_resnext50_32x4d.pth",
        ),
    }

    # Load the model from models dictionary
    model = models[model_name].model

    # Load the model state dictionary from local path
    state_dict = torch.load(models[model_name].url, map_location="cpu")["state_dict"]
    state_dict = rename_layers(state_dict, {"model.": ""})
    # Load the state dictionary into the model
    model.load_state_dict(state_dict)

    if activation == "softmax":
        return nn.Sequential(model, nn.Softmax(dim=1))

    return model

In [ ]:
orientation_model = create_model("swsl_resnext50_32x4d")
orientation_model.eval()

/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name swsl_resnext50_32x4d to current resnext50_32x4d.fb_swsl_ig1b_ft_in1k.
  model = create_fn(


Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Identity()
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, e

In [ ]:
def rotate_image(image, degrees):
    """
    Rotate a Torch tensor representing an image by a specified angle along dimensions 2 and 3.

    Args:
        image (torch.Tensor): Input image tensor.
        degrees (int): Rotation angle in degrees (90, 180, or 270).

    Returns:
        torch.Tensor: Rotated image tensor.
    """
    if degrees not in [90, 180, 270]:
        raise ValueError("Rotation angle must be 90, 180, or 270 degrees.")

    # Calculate the number of times to rotate the tensor by 90 degrees
    k = degrees // 90

    # Rotate the image tensor along dimensions 2 and 3
    rotated_image = torch.rot90(image, k=k, dims=(2, 3))

    return rotated_image

In [ ]:
def correct_orientation(image,path):

        if DEBUG:
            plt.imshow(image.squeeze(0).permute(1, 2, 0).numpy())
            plt.axis('off')  # Hide axis
            plt.show()

        resized_image = F.interpolate(image, size=(224, 224), mode='bilinear', align_corners=False)

        ####################
        with torch.no_grad():
            prediction = orientation_model(resized_image).numpy()[0]

        predictedRotationDeg = 360 - (prediction.argmax() * 90)

        if predictedRotationDeg == 360:
            print(f'{path}`s orientation is correct.')
        else:
            print(f'{path}`s ROTATED.')
            image = rotate_image(image, predictedRotationDeg)
        ####################

        if DEBUG:
            plt.imshow(image.squeeze(0).permute(1, 2, 0).numpy())
            plt.axis('off')  # Hide axis
            plt.show()

        return image

## Rotation Correction
In this section, we correct the rotation of the images.

<a id="1"></a>
# Finding image pairs

To find pairs of similar images, we use [DINOv2](https://arxiv.org/pdf/2304.07193.pdf) to obtain normalized image embeddings.

<center><img src="https://www.labellerr.com/blog/content/images/2023/05/Dino-v2-20230419.jpg" alt="DINOv2 example"></center>
Then, we calculate the distances between all the embeddings, and only keep those below a given distance threshold. For images with less than a set minimum number of pairs, the closest ones are kept instead.

In [ ]:
def embed_images(
    paths: list[Path],
    model_name: str,
    device: torch.device = torch.device("cpu"),
) -> T:
    """Computes image embeddings.

    Returns a tensor of shape [len(filenames), output_dim]
    """
    processor = AutoImageProcessor.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).eval().to(device)

    embeddings = []

    for i, path in tqdm(enumerate(paths), desc="Global descriptors"):
        image = load_torch_image(path)
        print(image.shape)
        ########################################################################################################################
        image = correct_orientation(image,path)
        ########################################################################################################################
        with torch.inference_mode():
            inputs = processor(images=image, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs) # last_hidden_state and pooled

            # Max pooling over all the hidden states but the first (starting token)
            # To obtain a tensor of shape [1, output_dim]
            # We normalize so that distances are computed in a better fashion later
            embedding = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=-1, p=2)

        embeddings.append(embedding.detach().cpu())
    return torch.cat(embeddings, dim=0)

In [ ]:
def get_pairs_exhaustive(lst: list[Any]) -> list[tuple[int, int]]:
    """Obtains all possible index pairs of a list"""
    return list(itertools.combinations(range(len(lst)), 2))

def get_image_pairs(
    paths: list[Path],
    model_name: str,
    similarity_threshold: float = 0.6,
    tolerance: int = 1000,
    min_matches: int = 20,
    exhaustive_if_less: int = 20,
    p: float = 2.0,
    device: torch.device = torch.device("cpu"),
) -> list[tuple[int, int]]:
    """Obtains pairs of similar images"""
    if len(paths) <= exhaustive_if_less:
        print("Do exhaustive pair")
        return get_pairs_exhaustive(paths)

    print(f'Use image embed dist pair with similarity_threshold = {similarity_threshold}')
    matches = []

    # Embed images and compute distances for filtering
    embeddings = embed_images(paths, model_name)
    distances = torch.cdist(embeddings, embeddings, p=p)

    # Remove pairs above similarity threshold (if enough)
    mask = distances <= similarity_threshold
    image_indices = np.arange(len(paths))

    for current_image_index in range(len(paths)):
        mask_row = mask[current_image_index]
        indices_to_match = image_indices[mask_row]

        # We don't have enough matches below the threshold, we pick most similar ones
        if len(indices_to_match) < min_matches:
            indices_to_match = np.argsort(distances[current_image_index])[:min_matches]

        for other_image_index in indices_to_match:
            # Skip an image matching itself
            if other_image_index == current_image_index:
                continue

            # We need to check if we are below a certain distance tolerance
            # since for images that don't have enough matches, we picked
            # the most similar ones (which could all still be very different
            # to the image we are analyzing)
            if distances[current_image_index, other_image_index] < tolerance:
                # Add the pair in a sorted manner to avoid redundancy
                matches.append(tuple(sorted((current_image_index, other_image_index.item()))))

    return sorted(list(set(matches)))

In [ ]:
# if JEANS:
#     images_list = list(Path("/kaggle/input/image-matching-challenge-2024/test/church/images/").glob("*.png"))[:30]
#     index_pairs = get_image_pairs(images_list, "/kaggle/input/dinov2/pytorch/base/1")
#     print(index_pairs)

if DEBUG:
    images_list = list(Path("/kaggle/input/image-matching-challenge-2024/train/dioscuri/images").glob("*.png"))[:30]
    index_pairs = get_image_pairs(images_list, "/kaggle/input/dinov2/pytorch/base/1")
    print(index_pairs)

<a id="2"></a>
# Computing keypoints

In order to be able to know the position of each camera, we must be able to relate images to each other. For this, we extract relevant keypoints and compare pairs of image keypoints against each other. There are many ways to extract relevant keypoints, the most traditional one being [SIFT](https://en.wikipedia.org/wiki/Scale-invariant_feature_transform). However, newer and improved methods exist now, one of which is [ALIKED](https://arxiv.org/abs/2304.03608), the keypoint extraction method used in the solution.

<center><img src="https://www.catalyzex.com/_next/image?url=https%3A%2F%2Fd3i71xaburhd42.cloudfront.net%2Faf9fc17471b4c38211c3d9f5058c9c1f59501eea%2F3-Figure1-1.png&w=640&q=75" alt="ALIKED architecture"></center>


Let's take a closer look at the keypoints that ALIKED extracts.

In [ ]:
if DEBUG:
    dtype = torch.float32 # ALIKED has issues with float16

    extractor = ALIKED(
            max_num_keypoints=4096,
            detection_threshold=0.01,
            resize=1024
        ).eval().to(device, dtype)

    path = images_list[0]
    image = load_torch_image(path, device=device).to(dtype)
    features = extractor.extract(image)

    fig, ax = plt.subplots(1, 2, figsize=(10, 20))
    ax[0].imshow(image[0, ...].permute(1,2,0).cpu())
    ax[1].imshow(image[0, ...].permute(1,2,0).cpu())
    ax[1].scatter(features["keypoints"][0, :, 0].cpu(), features["keypoints"][0, :, 1].cpu(), s=0.5, c="red")

    del extractor

In [ ]:
def detect_keypoints(
    paths: list[Path],
    feature_dir: Path,
    num_features: int = 4096,
    resize_to: int = 1024,
    device: torch.device = torch.device("cpu"),
) -> None:
    """Detects the keypoints in a list of images with ALIKED

    Stores them in feature_dir/keypoints.h5 and feature_dir/descriptors.h5
    to be used later with LightGlue
    """
    dtype = torch.float32 # ALIKED has issues with float16

    extractor = ALIKED(
        max_num_keypoints=num_features,
        detection_threshold=0.01,
        resize=resize_to
    ).eval().to(device, dtype)

    feature_dir.mkdir(parents=True, exist_ok=True)

    with h5py.File(feature_dir / "keypoints.h5", mode="w") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="w") as f_descriptors:

        for path in tqdm(paths, desc="Computing keypoints"):
            key = path.name

            with torch.inference_mode():
                image = load_torch_image(path, device=device).to(dtype)
                features = extractor.extract(image)

                f_keypoints[key] = features["keypoints"].squeeze().detach().cpu().numpy()
                f_descriptors[key] = features["descriptors"].squeeze().detach().cpu().numpy()

In [ ]:
if DEBUG:
    feature_dir = Path("./sample_test_features")
    detect_keypoints(images_list, feature_dir)

<a id="3"></a>
# Match and compute keypoint distances

Now that we have the relevant image pairs and keypoints, we can go ahead and compare the keypoints of the images in a pair to find a good relationship between them. This is done with [LightGlue](https://arxiv.org/abs/2306.13643), which matches the keypoints and their descriptors between two images.

<center><img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fraw.githubusercontent.com%2Fcvg%2Flightglue%2Fmaster%2Fassets%2Feasy_hard.jpg&f=1&nofb=1&ipt=60962b56b05d3e8f95a064ab2a6010e5a6cbd5f1d10379d90e660b2561a3bae9&ipo=images" alt="LightGlue example"></center>

In [ ]:
if DEBUG:
    matcher_params = {
        "width_confidence": -1,
        "depth_confidence": -1,
        "mp": True if 'cuda' in str(device) else False,
    }
    matcher = KF.LightGlueMatcher("aliked", matcher_params).eval().to(device)

    with h5py.File(feature_dir / "keypoints.h5", mode="r") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="r") as f_descriptors:
            idx1, idx2 = index_pairs[0]
            key1, key2 = images_list[idx1].name, images_list[idx2].name

            keypoints1 = torch.from_numpy(f_keypoints[key1][...]).to(device)
            keypoints2 = torch.from_numpy(f_keypoints[key2][...]).to(device)
            print("Keypoints:", keypoints1.shape, keypoints2.shape)
            descriptors1 = torch.from_numpy(f_descriptors[key1][...]).to(device)
            descriptors2 = torch.from_numpy(f_descriptors[key2][...]).to(device)
            print("Descriptors:", descriptors1.shape, descriptors2.shape)

            with torch.inference_mode():
                distances, indices = matcher(
                    descriptors1,
                    descriptors2,
                    KF.laf_from_center_scale_ori(keypoints1[None]),
                    KF.laf_from_center_scale_ori(keypoints2[None]),
                )
    print(distances, indices)

In [ ]:
def keypoint_distances(
    paths: list[Path],
    index_pairs: list[tuple[int, int]],
    feature_dir: Path,
    min_matches: int = 15,
    verbose: bool = True,
    device: torch.device = torch.device("cpu"),
) -> None:
    """Computes distances between keypoints of images.

    Stores output at feature_dir/matches.h5
    """

    matcher_params = {
        "width_confidence": -1,
        "depth_confidence": -1,
        "mp": True if 'cuda' in str(device) else False,
    }
    matcher = KF.LightGlueMatcher("aliked", matcher_params).eval().to(device)

    with h5py.File(feature_dir / "keypoints.h5", mode="r") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="r") as f_descriptors, \
         h5py.File(feature_dir / "matches.h5", mode="w") as f_matches:

            for idx1, idx2 in tqdm(index_pairs, desc="Computing keypoing distances"):
                key1, key2 = paths[idx1].name, paths[idx2].name

                keypoints1 = torch.from_numpy(f_keypoints[key1][...]).to(device)
                keypoints2 = torch.from_numpy(f_keypoints[key2][...]).to(device)
                descriptors1 = torch.from_numpy(f_descriptors[key1][...]).to(device)
                descriptors2 = torch.from_numpy(f_descriptors[key2][...]).to(device)

                with torch.inference_mode():
                    distances, indices = matcher(
                        descriptors1,
                        descriptors2,
                        KF.laf_from_center_scale_ori(keypoints1[None]),
                        KF.laf_from_center_scale_ori(keypoints2[None]),
                    )

                # We have matches to consider
                n_matches = len(indices)
                if n_matches:
                    if verbose:
                        print(f"{key1}-{key2}: {n_matches} matches")
                    # Store the matches in the group of one image
                    if n_matches >= min_matches:
                        group  = f_matches.require_group(key1)
                        group.create_dataset(key2, data=indices.detach().cpu().numpy().reshape(-1, 2))

In [ ]:
if DEBUG:
    keypoint_distances(images_list, index_pairs, feature_dir, verbose=False)

<a id="4"></a>
# RANSAC
Up to now, we have matched keypoints and their descriptors extracted from pairs of images. This is described by a [fundamental matrix](https://en.wikipedia.org/wiki/Fundamental_matrix_(computer_vision)) denoted as $F$. In epipolar geometry, with homogeneous image coordinates, $x$ and $x′$, of corresponding points in a stereo image pair, $Fx$ describes a line (an epipolar line) on which the corresponding point $x′$ on the other image must lie. That means, for all pairs of corresponding points, $x'Fx = 0$ holds. This is known as epipolar constraint or correspondance condition (or Longuet-Higgins equation), and is solved via the [eight-point algorithm](https://en.wikipedia.org/wiki/Eight-point_algorithm).

<center><img src="https://cmsc426.github.io/assets/sfm/epipole1.png" alt="Fundamental matrix"></center>

Since the keypoint correspondences are computed using feature descriptors, the data is bound to be noisy and (in general) contains several outliers. Thus, to remove these outliers, we use a [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus) algorithm to find the best possible fundamental matrix. So, out of all possibilities, the $F$ matrix with maximum number of inliers is chosen.

<center><img src="https://cmsc426.github.io/assets/sfm/ransac.png" alt="RANSAC"></center>

In [ ]:
def import_into_colmap(
    path: Path,
    feature_dir: Path,
    database_path: str = "colmap.db",
) -> None:
    """Adds keypoints into colmap"""
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, path, "", "simple-pinhole", single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()

In [ ]:
if DEBUG:
    database_path = "colmap.db"
    images_dir = images_list[0].parent
    import_into_colmap(
        images_dir,
        feature_dir,
        database_path,
    )

    # This does RANSAC
    pycolmap.match_exhaustive(database_path)

<a id="5"></a>
# Sparse Reconstruction

Now we have similar image pairs, with matched keypoint descriptors, without outliers! All that is left is to construct the scene and obtain the camera positions. We do this with pycolmap, which offers an incremental reconstruction algorithm that starts from two pairs of images and continually adds more and more images to the scene, resulting in a reconstructed scene with camera information. We can then use the camera rotation and translation as our submission!

In [ ]:
if DEBUG:
    mapper_options = pycolmap.IncrementalPipelineOptions()
    mapper_options.min_model_size = 3
    mapper_options.max_num_models = 2

    maps = pycolmap.incremental_mapping(
        database_path=database_path,
        image_path=images_dir,
        output_path=Path.cwd() / "incremental_pipeline_outputs",
        options=mapper_options,
    )

In [ ]:
if DEBUG:
    print(maps[0].summary())
    for k, im in maps[0].images.items():
        print("Rotation", im.cam_from_world.rotation.matrix(), "Translation:", im.cam_from_world.translation, sep="\n")
        print()

# Running everything

In [ ]:
def parse_sample_submission(
    base_path: Path,
) -> dict[dict[str, list[Path]]]:
    """Construct a dict describing the test data as

    {"dataset": {"scene": [<image paths>]}}
    """
    data_dict = {}
    with open(base_path / "sample_submission.csv", "r") as f:
        for i, l in enumerate(f):
            # Skip header
            if i == 0:
                print("header:", l)

            if l and i > 0:
                image_path, dataset, scene, _, _ = l.strip().split(',')
                if dataset not in data_dict:
                    data_dict[dataset] = {}
                if scene not in data_dict[dataset]:
                    data_dict[dataset][scene] = []
                data_dict[dataset][scene].append(Path(base_path / image_path))

    for dataset in data_dict:
        for scene in data_dict[dataset]:
            print(f"{dataset} / {scene} -> {len(data_dict[dataset][scene])} images")

    return data_dict

In [ ]:
def create_submission(
    results: dict,
    data_dict: dict[dict[str, list[Path]]],
    base_path: Path,
) -> None:
    """Prepares a submission file."""

    with open("submission.csv", "w") as f:
        f.write("image_path,dataset,scene,rotation_matrix,translation_vector\n")

        for dataset in data_dict:
            # Only write results for datasets with images that have results
            if dataset in results:
                res = results[dataset]
            else:
                res = {}

            # Same for scenes
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}

                # Write the row with rotation and translation matrices
                for image in data_dict[dataset][scene]:
                    if image in scene_res:
                        print(image)
                        R = scene_res[image]["R"].reshape(-1)
                        T = scene_res[image]["t"].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    image_path = str(image.relative_to(base_path))
                    f.write(f"{image_path},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n")

In [ ]:
class Config:
    base_path: Path = Path("/kaggle/input/image-matching-challenge-2024")
    feature_dir: Path = Path.cwd() / ".feature_outputs"

    device: torch.device = K.utils.get_cuda_device_if_available(0)

    pair_matching_args = {
        "model_name": "/kaggle/input/dinov2/pytorch/base/1",
        "similarity_threshold": 0.3,
        "tolerance": 500,
        "min_matches": 50,
        "exhaustive_if_less": 50,
        "p": 2.0,
    }

    keypoint_detection_args = {
        "num_features": 4096,
        "resize_to": 1024,
    }

    keypoint_distances_args = {
        "min_matches": 15,
        "verbose": False,
    }

    colmap_mapper_options = {
        "min_model_size": 3, # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
        "max_num_models": 2,
    }

In [ ]:
def run_from_config(config: Config) -> None:
    results = {}

    data_dict = parse_sample_submission(config.base_path)
    datasets = list(data_dict.keys())

    for dataset in datasets:
        if dataset not in results:
            results[dataset] = {}

        for scene in data_dict[dataset]:
            images_dir = data_dict[dataset][scene][0].parent
            results[dataset][scene] = {}
            image_paths = data_dict[dataset][scene]
            print (f"Got {len(image_paths)} images")

            try:
                feature_dir = config.feature_dir / f"{dataset}_{scene}"
                feature_dir.mkdir(parents=True, exist_ok=True)
                database_path = feature_dir / "colmap.db"
                if database_path.exists():
                    database_path.unlink()

                # 1. Get the pairs of images that are somewhat similar
                index_pairs = get_image_pairs(
                    image_paths,
                    **config.pair_matching_args,
                    device=config.device,
                )
                gc.collect()

                # 2. Detect keypoints of all images
                detect_keypoints(
                    image_paths,
                    feature_dir,
                    **config.keypoint_detection_args,
                    device=device,
                )
                gc.collect()

                # 3. Match  keypoints of pairs of similar images
                keypoint_distances(
                    image_paths,
                    index_pairs,
                    feature_dir,
                    **config.keypoint_distances_args,
                    device=device,
                )
                gc.collect()

                sleep(1)

                # 4.1. Import keypoint distances of matches into colmap for RANSAC
                import_into_colmap(
                    images_dir,
                    feature_dir,
                    database_path,
                )

                output_path = feature_dir / "colmap_rec_aliked"
                output_path.mkdir(parents=True, exist_ok=True)

                # 4.2. Compute RANSAC (detect match outliers)
                # By doing it exhaustively we guarantee we will find the best possible configuration
                pycolmap.match_exhaustive(database_path)

                mapper_options = pycolmap.IncrementalPipelineOptions(**config.colmap_mapper_options)

                # 5.1 Incrementally start reconstructing the scene (sparse reconstruction)
                # The process starts from a random pair of images and is incrementally extended by
                # registering new images and triangulating new points.
                maps = pycolmap.incremental_mapping(
                    database_path=database_path,
                    image_path=images_dir,
                    output_path=output_path,
                    options=mapper_options,
                )

                print(maps)
                clear_output(wait=False)

                # 5.2. Look for the best reconstruction: The incremental mapping offered by
                # pycolmap attempts to reconstruct multiple models, we must pick the best one
                images_registered  = 0
                best_idx = None

                print ("Looking for the best reconstruction")

                if isinstance(maps, dict):
                    for idx1, rec in maps.items():
                        print(idx1, rec.summary())
                        try:
                            if len(rec.images) > images_registered:
                                images_registered = len(rec.images)
                                best_idx = idx1
                        except Exception:
                            continue

                # Parse the reconstruction object to get the rotation matrix and translation vector
                # obtained for each image in the reconstruction
                if best_idx is not None:
                    for k, im in maps[best_idx].images.items():
                        key = config.base_path / "test" / scene / "images" / im.name
                        results[dataset][scene][key] = {}
                        results[dataset][scene][key]["R"] = deepcopy(im.cam_from_world.rotation.matrix())
                        results[dataset][scene][key]["t"] = deepcopy(np.array(im.cam_from_world.translation))

                print(f"Registered: {dataset} / {scene} -> {len(results[dataset][scene])} images")
                print(f"Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images")
                create_submission(results, data_dict, config.base_path)
                gc.collect()

            except Exception as e:
                print(e)

In [ ]:
run_from_config(Config)

Looking for the best reconstruction
0 Reconstruction:
	num_reg_images = 40
	num_cameras = 40
	num_points3D = 12282
	num_observations = 64243
	mean_track_length = 5.23066
	mean_observations_per_image = 1606.08
	mean_reprojection_error = 0.913796
Registered: church / church -> 40 images
Total: church / church -> 41 images
/kaggle/input/image-matching-challenge-2024/test/church/images/00046.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00090.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00092.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00087.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00050.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00068.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00083.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00096.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00069.png
/kaggle/input/image-m

In [ ]:
!cat submission.csv

image_path,dataset,scene,rotation_matrix,translation_vector
test/church/images/00046.png,church,church,-0.21696721375931327;-0.34027467987547655;-0.9149526601902211;0.36251374604519193;0.8421679060855337;-0.3991703920480447;0.9063713433697043;-0.41828980407744776;-0.059368575214905794,5.92333490303866;5.138910148082694;11.502384010895256
test/church/images/00090.png,church,church,0.9985527260440824;-0.0009035806843229544;-0.05377394212700625;-0.008632023306604147;0.9842035839927694;-0.17682984315273784;0.05308428659750399;0.17703809984784216;0.9827713720488095,0.0756176888953241;-0.010867732971685288;-0.7389864528026447
test/church/images/00092.png,church,church,0.9387787544044988;-0.1234343294188164;-0.32164952448226136;0.14406819979236563;0.9887155725046378;0.04105935332434388;0.31295176000036296;-0.08488511652671157;0.9459681352481816,0.25265126560114204;-0.10606084686745151;-0.5526490094356777
test/church/images/00087.png,church,church,0.8817982779404965;-0.16375409712047986;-0.442

# What to do next?

Here are some ways in which you can explore potential improvements:

- Using a different image embedding model to obtain the image pairs
- Trying other approaches for keypoint extraction, such as SIFT or DISK
- Leveraging the training data to train a better models for each dataset